In [101]:
import pandas as pd
import numpy as np
import openpyxl

In [102]:
months = ["Ноябрь 2022", "Декабрь 2022", "Январь 2023", "Февраль 2023", "Март 2023", "Апрель 2023", "Май 2023", "Июнь 2023", "Июль 2023", "Август 2023", "Сентябрь 2023", "Октябрь 2023", "Ноябрь 2023", "Декабрь 2023", "Январь 2024", "Февраль 2024"]
month_to_index = {m: i for i, m in enumerate(months)}

In [103]:
df_prol = pd.read_csv("prolongations.csv", sep=",")
df_fin = pd.read_csv("financial_data.csv", sep=",")

In [104]:
df_prol.head(5)

,id,month,AM
0,42,ноябрь 2022,Васильев Артем Александрович
1,453,ноябрь 2022,Васильев Артем Александрович
2,548,ноябрь 2022,Михайлов Андрей Сергеевич
3,87,ноябрь 2022,Соколова Анастасия Викторовна
4,429,ноябрь 2022,Соколова Анастасия Викторовна


In [105]:
df_fin.head(5)

,id,Причина дубля,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,Account
0,42,NaN,"36 220,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
1,657,первая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
2,657,вторая часть оплаты,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
3,594,NaN,стоп,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович
4,665,NaN,"10 000,00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Васильев Артем Александрович


In [106]:
counter = 0
for ind in df_prol['id'].unique():
    if len(df_prol[df_prol['id'] == ind]) >= 2:
        print(df_prol[df_prol['id'] == ind])
        counter += 1
    if counter == 5:
        break

      id         month                            AM
1    453   ноябрь 2022  Васильев Артем Александрович
113  453  февраль 2023  Васильев Артем Александрович
     id        month                         AM
2   548  ноябрь 2022  Михайлов Андрей Сергеевич
93  548  январь 2023  Михайлов Андрей Сергеевич
     id        month                             AM
3    87  ноябрь 2022  Соколова Анастасия Викторовна
232  87    июнь 2023        Иванова Мария Сергеевна
      id         month                             AM
4    429   ноябрь 2022  Соколова Анастасия Викторовна
114  429  февраль 2023  Соколова Анастасия Викторовна
      id         month                            AM
5    600   ноябрь 2022  Васильев Артем Александрович
117  600  февраль 2023  Васильев Артем Александрович
186  600   апрель 2023  Васильев Артем Александрович
187  600   апрель 2023  Васильев Артем Александрович
233  600     июнь 2023  Васильев Артем Александрович


In [107]:
df_prol = df_prol.drop_duplicates(subset=['id'], keep='last') # я предположил, что несколько записей об одном проекте - просто история проекта, поэтому актуальной записью будем считать последнюю
for ind in df_prol['id'].unique():
    if len(df_prol[df_prol['id'] == ind]) >= 2:
        print(df_prol[df_prol['id'] == ind])

In [108]:
len(df_prol['id'].unique())

313

In [109]:
len(df_fin['id'].unique()) # значит об одном проекте нет информации о месяце завершения, поэтому не будем его рассматривать

314

In [110]:
for ind in df_fin['id'].unique():
    if ind not in df_prol['id'].unique():
        df_fin = df_fin.drop(df_fin[df_fin['id'] == ind].index)

In [111]:
df_fin['id'].unique().sort() == df_prol['id'].unique().sort()

True

In [112]:
def format_month(month):
    return month.strip().title()

In [113]:
# убираем лишнюю колонку менеджеров, форматируем колонку месяца завершения для дальнейшего использования, заполнение пропусков
df = pd.merge(df_fin, df_prol, suffixes=['_df1', '_df2'], how="left", on="id")
df = df.drop(["Account"], axis=1)
df['month'] = df['month'].apply(format_month)
for month in months:
    df[month] = df[month].fillna(0)
    df[month] = df[month].replace({'в ноль': '0'})

In [114]:
# удаление записей согласно правилам стоп и end
mask = df[months].isin(['стоп', 'end']).any(axis=1)
df = df[~mask].copy()
df[months] = df[months].replace({'стоп': '0', 'end': '0'})

In [115]:
def to_float(value):
    if pd.isna(value) or value == 0 or value == '0':
        return 0.0
    string = str(value)
    string = string.replace('\xa0', '').replace(' ', '').replace(',', '.')
    return float(string)

In [116]:
# приведение к типу float
for month in months:
    df[month] = df[month].apply(to_float)

In [117]:
df.drop(["Причина дубля"], axis=1, inplace=True)

In [118]:
# агрегация записей по id проекта
agg_dict = {col: 'sum' for col in months}
agg_dict.update({col: 'first' for col in df.columns if col not in months and col != 'id'})
df = df.groupby('id', as_index=False).agg(agg_dict)
df

,id,Ноябрь 2022,Декабрь 2022,Январь 2023,Февраль 2023,Март 2023,Апрель 2023,Май 2023,Июнь 2023,Июль 2023,Август 2023,Сентябрь 2023,Октябрь 2023,Ноябрь 2023,Декабрь 2023,Январь 2024,Февраль 2024,month,AM
0,15,439280.0,439280.0,102433.75,102433.75,102433.75,138158.0,138158.0,102433.75,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Июнь 2023,Иванова Мария Сергеевна
1,16,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Декабрь 2022,Иванова Мария Сергеевна
2,31,55100.0,55100.0,0.00,44775.00,44775.00,44775.0,44775.0,44775.00,44775.0,44775.0,44775.0,44775.0,44775.0,44775.00,44775.0,46200.0,Декабрь 2022,Васильев Артем Александрович
3,39,137700.0,137700.0,149206.50,149206.50,149206.50,149206.5,149206.5,149206.50,149206.5,149206.5,149206.5,149206.5,149206.5,149206.50,0.0,0.0,Декабрь 2023,Попова Екатерина Николаевна
4,42,36220.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,Ноябрь 2022,Васильев Артем Александрович
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,1001,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,290000.0,0.00,0.0,0.0,Ноябрь 2023,Кузнецов Михаил Иванович
259,1004,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,34000.0,0.00,0.0,0.0,Декабрь 2023,без А/М
260,1006,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,145195.0,140495.0,140495.00,0.0,0.0,Декабрь 2023,Смирнова Ольга Владимировна
261,1012,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,98492.0,109442.52,0.0,0.0,Ноябрь 2023,Петрова Анна Дмитриевна


In [119]:
df.to_csv("df.csv", sep=",", index=False)

In [120]:
df_adj = df.copy()
# обработка правила в ноль
for i, row in df_adj.iterrows():
    values = row[months].astype(float).values.copy()
    if not np.any(values):
        continue
    for j in range(1, len(months)):
        if values[j] == 0 and np.any(values[:j] != 0):
            last_nonzero = values[:j][values[:j] != 0][-1]
            values[j] = last_nonzero
    df_adj.loc[i, months] = values


In [121]:
def calc_prolongation(df):
    results = []
    for month in months[:-2]:
        m_idx = month_to_index[month]
        next_m = months[m_idx + 1]
        next2_m = months[m_idx + 2]

        finished = df[df['month'] == month].copy()
        if finished.empty:
            results.append({
                'месяц завершения': month,
                'месяц пролонгации 1': next_m,
                'месяц пролонгации 2': next2_m,
                'coef1': np.nan,
                'coef2': np.nan
            })
            continue

        base_sum = finished[month].sum()
        first_ext = finished[next_m].sum()
        coef1 = first_ext / base_sum if base_sum != 0 else np.nan

        not_first = finished[finished[next_m] == finished[month]]
        base_sum_2 = not_first[month].sum()
        second_ext = not_first[next2_m].sum()
        coef2 = second_ext / base_sum_2 if base_sum_2 != 0 else np.nan

        results.append({
            'месяц завершения': month,
            'месяц пролонгации 1': next_m,
            'месяц пролонгации 2': next2_m,
            'coef1': coef1,
            'coef2': coef2
        })
    return pd.DataFrame(results)


In [122]:
manager_results = []
for am, subdf in df_adj.groupby('AM'):
    res = calc_prolongation(subdf)
    res['AM'] = am
    manager_results.append(res)

manager_df = pd.concat(manager_results, ignore_index=True)

dept_df = calc_prolongation(df_adj)
dept_df['AM'] = 'Отдел в целом'

final = pd.concat([manager_df, dept_df], ignore_index=True)

annual = (
    final.groupby('AM')[['coef1', 'coef2']]
    .mean()
    .reset_index()
    .rename(columns={
        'coef1': 'Годовой коэффициент (1-й мес)',
        'coef2': 'Годовой коэффициент (2-й мес)'
    })
)

In [123]:
with pd.ExcelWriter("report.xlsx", engine="openpyxl") as writer:
    manager_df.to_excel(writer, sheet_name="По менеджерам", index=False)
    dept_df.to_excel(writer, sheet_name="Отдел в целом", index=False)
    annual.to_excel(writer, sheet_name="Годовые коэффициенты", index=False)

    workbook = writer.book
    for sheet_name in writer.sheets:
        worksheet = writer.sheets[sheet_name]
        for col in worksheet.columns:
            max_length = 0
            col_letter = col[0].column_letter
            for cell in col:
                try:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
                except:
                    pass
            worksheet.column_dimensions[col_letter].width = max_length + 2